# 2. Orbital basis rotation

This notebook shows how to relate an orbital basis change to the corresponding unitary transformation on the whole Fock space.

In [1]:
import fermi_relations as fr
import numpy as np
from scipy.linalg import expm
import scipy.sparse.linalg as spla

In [2]:
# random number generator
rng = np.random.default_rng(43)

Let us first define an orbital basis rotation via the unitary matrix
\begin{equation*}
u = e^{-i h}
\end{equation*}
where $h \in \mathbb{C}^{L \times L}$ is Hermitian.

In [3]:
# number of modes (or orbitals) L
nmodes = 7

# random single-particle base change matrix as matrix exponential
h = fr.crandn((nmodes, nmodes), rng)
h = 0.5*(h + h.conj().T)
u = expm(-1j*h)

# 'u' is indeed unitary
np.linalg.norm(u.conj().T @ u - np.identity(nmodes))

np.float64(1.4612831771713532e-15)

The corresponding base change matrix on the whole Fock space is then
\begin{equation*}
U = e^{-i H} \quad \text{with} \quad H = \sum_{j,k=0}^{L-1} h_{jk} a_j^{\dagger} a_k
\end{equation*}

In [4]:
clist, alist, _ = fr.construct_fermionic_operators(nmodes)
hfock = sum(h[i, j] * (clist[i] @ alist[j])
            for i in range(nmodes)
            for j in range(nmodes))
ufock = expm(-1j * hfock.toarray())

# 'ufock' is indeed unitary
print(np.linalg.norm(ufock.conj().T @ ufock - np.identity(2**nmodes)))

# alternative construction of base change matrix on whole Fock space
ufock_alt = fr.fock_orbital_base_change(u)
np.allclose(ufock, ufock_alt.toarray())

7.22158192829309e-15


True

Conjugating a fermionic operator by $U$ corresponds to an orbital basis change:
\begin{equation*}
U a_{\varphi}^{\dagger} U^{\dagger} = a_{u \varphi}^{\dagger}
\end{equation*}
where $u \varphi$ is the usual matrix-vector product.

This relation is a consequence of the Campbell identity
\begin{equation*}
\mathrm{Ad}_{e^{-i H}} = e^{\mathrm{ad}_{-i H}}
\end{equation*}
where $\mathrm{Ad}_{e^X} Y = e^X Y e^{-X}$, $\mathrm{ad}_X Y = [X, Y]$, and
\begin{equation*}
e^{\mathrm{ad}_X} Y = Y + [X, Y] + \frac{1}{2!} [X, [X, Y]] + \dots
\end{equation*}
together with
\begin{equation*}
\big[H, a_{\varphi}^{\dagger}\big] = \Bigg[\sum_{j,k=0}^{L-1} h_{jk} a_j^{\dagger} a_k, a_{\varphi}^{\dagger}\Bigg] = a_{h \varphi}^{\dagger}.
\end{equation*}

In [5]:
# define a random "orbital" state
phi = fr.crandn(nmodes, rng)
phi /= np.linalg.norm(phi)

# numerically verify the equations above
print(np.linalg.norm(ufock @ fr.orbital_create_op(phi) @ ufock.conj().T - fr.orbital_create_op(u @ phi)))
print(np.linalg.norm(ufock @ fr.orbital_annihil_op(phi) @ ufock.conj().T - fr.orbital_annihil_op(u @ phi)))

print(spla.norm(fr.comm(hfock, fr.orbital_create_op(phi)) - fr.orbital_create_op(h @ phi)))

6.2572486873618715e-15
6.2862960161376585e-15
3.063032439280136e-15
